In [34]:
import glob
import re
import cv2
import skimage.exposure
import pandas as pd

In [17]:
location = "."

files = sorted([f for f in glob.glob(location+"/**", recursive = True) if re.search(r'(.*)\.(jpg|jpeg)$', f)])   
print(files)

['./African_Bush_Elephant.jpg', './blur.jpg', './jack-russell-blurred.jpg', './stop-taking-blurry-pictures.jpg']


In [31]:
def blur_contrast_brightness_measure(imagePath):
    image = cv2.imread(imagePath)
    blur_measure = cv2.Laplacian(image, cv2.CV_64F).var()
    
    low_contrast = False
    threshold = 0
    while not low_contrast:
        threshold = round(threshold + 0.025, 3)
        low_contrast = skimage.exposure.is_low_contrast(image, fraction_threshold=threshold)
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    return [round(blur_measure, 3), threshold, round(hsv[...,2].mean(),3)]

In [32]:
blur_contrast_brightness_measure(files[0])

[1100.174, 0.45, 169.472]

In [59]:
df = pd.DataFrame(columns = ['File_name', 'Blur_Measure', 'Contrast_Measure', 'Brightness_Measure']) 
for i in range(0, len(files)):
    filec = files[i]
    measure = blur_contrast_brightness_measure(filec)
    print(measure)
    df = df.append(pd.Series([filec]+measure, index=df.columns), ignore_index = True)

[1100.174, 0.45, 169.472]
[98.222, 0.475, 191.638]
[6.469, 0.425, 114.688]
[71.21, 0.45, 155.782]


In [60]:
df

,File_name,Blur_Measure,Contrast_Measure,Brightness_Measure
0,./African_Bush_Elephant.jpg,1100.174,0.450,169.472
1,./blur.jpg,98.222,0.475,191.638
2,./jack-russell-blurred.jpg,6.469,0.425,114.688
3,./stop-taking-blurry-pictures.jpg,71.210,0.450,155.782


In [61]:
df.to_csv("image_stats.csv", index = False)